In [ ]:
from langchain.llms import OpenAI

In [ ]:
from dotenv import load_dotenv
import os
load_dotenv()
key = os.environ["OPENAI_API_KEY"]
llm =OpenAI(api_key=key,temperature=0.6)

In [ ]:
text ="What is the capital of India"
print(llm.predict(text))
os.environ["HUGGINGFACEHUB_API_TOKEN"]="<HuggingFaceKey>print(os.environ["HUGGINGFACEHUB_API_TOKEN"])

In [ ]:
from langchain import HuggingFaceHub
llm_huggingface= HuggingFaceHub(repo_id="google/flan-t5-large",model_kwargs={"temperature":0,"max_length":64})

In [ ]:
output = llm_huggingface.predict("What is capital of Russia?")
print(output)

Prompt Templates

In [ ]:
from langchain.prompts import PromptTemplate
prompt_template = PromptTemplate(input_variables=['country'],template ="Tell me capital of this country: {country}")
prompt_template.format(country="India")

In [ ]:
from langchain.chains import LLMChain
chain = LLMChain(llm=llm, prompt=prompt_template)
print(chain.run("[India, USA]"))

In [ ]:
from langchain.prompts import PromptTemplate
capital_template = PromptTemplate(input_variables=['country'],template ="Tell me capital of this country: {country}")
capital_chain = LLMChain(llm=llm,prompt=capital_template,output_key="capital")
famous_template = PromptTemplate(input_variables=['capital'],template ="Suggest me some amazing places of this city: {capital}")


In [ ]:
famous_chain = LLMChain(llm=llm,prompt=famous_template,output_key="places")
from langchain.chains import SimpleSequentialChain
chain = SimpleSequentialChain(chains = [capital_chain,famous_chain])
print(chain.run("India"))


In [ ]:
from langchain.chains import SequentialChain
chain = SequentialChain(chains = [capital_chain,famous_chain], input_variables=['country'],output_variables=['capital','places'] )


In [ ]:
chain({
    'country':"India, SriLanka"
})

CHAT OPenAI

In [ ]:
from langchain.chat_models import ChatOpenAI

In [ ]:
from langchain.schema import HumanMessage,SystemMessage,AIMessage
chatllm = ChatOpenAI(openai_api_key=os.environ["OPENAI_API_KEY"],temperature=0.6,model='gpt-3.5-turbo')


In [ ]:
chatllm([
    SystemMessage(content="You are a comedian AI assistant"),
    HumanMessage(content="Please provide me some war bollywood movies")
])

Prompt Template + LLM + OP Parsers

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate
from langchain.schema import BaseOutputParser

In [ ]:
class Commmaseperatedoutput(BaseOutputParser):
    def parse(self, text:str):
        return text

In [ ]:
template ="You are a helpful assistant. When user gives any input, you should generate 5 words synonyms !"
human_template = "{text}"
chatprompt = ChatPromptTemplate.from_messages([
    ("system", template),
    ("human", human_template)
])


In [ ]:
chain = chatprompt|chatllm|Commmaseperatedoutput()

In [ ]:
chain.invoke({"text":"sports"})